In [ ]:
import boto3
import pandas as pd
from io import stringIO #python3; python2: BytesIO

In [ ]:
AWS_ACCESS_KEY = "AKIATAMBR5AF2JM526U"
AWS_SECRET_KEY = "eTgtaB8qJ1/uZ7NvANOUpAN0DAU9Gl1U6IIooHPO"
AWS_REGION = "ap-south-1"
SCHEMA_NAME = "COVID_19"
S3_STAGING_DIR = "s3://gourav-test-bucket/output/"
S3_BUCKET_NAME = "gourav-test-bucket"
S3_OUTPUT_DIRECTORY = "output"

In [ ]:
athena_client = boto3.client(
    "athena",
    aws_access_key = AWS_ACCESS_KEY,
    aws_secret_access = AWS_SECRET_KEY,
    region_name = AWS_REGION,
)

In [ ]:
dict = {}

def download_and_load_query_result(
    client: boto3.client, query_response = dict
) -> pd.DataFrame:
    while True:
        try:
            # this function only loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId = query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
        temp_file_location: str = "athena_query_results.csv"
        s3_client = boto3.client (
            "s3",
            aws_access_key_id = AWS_ACCESS_KEY,
            aws_secret_access = AWS_SECRET_KEY,
            region_name = AWS_REGION
        )
        s3_client.download_file (
            S3_BUCKET_NAME,
            f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
            temp_file_location
        )
        return pd.read_csv(temp_file_location)

In [ ]:
response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM enigma_jhud",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

enigma_jhud = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM nytimes_data_in_usa_us_county",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

nytimes_data_in_usa_us_county = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM nytimes_data_in_usa_us_states",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

nytimes_data_in_usa_us_states = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM static_datacountrycode",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

static_datacountrycode = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM static_datacountypopulation",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

static_datacountypopulation = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM rearc_covid_19_testing_dataus_daily",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

rearc_covid_19_testing_dataus_daily = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM rearc_usa_hospital_beds",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

rearc_usa_hospital_beds = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM rearc_covid_19_testing_dataus_total_latest",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

rearc_covid_19_testing_dataus_total_latest = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM rearc_covid_19_testing_data_states_dailystates_daily",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

rearc_covid_19_testing_data_states_dailystates_daily = download_and_load_query_result(athena_client, response)

In [ ]:
response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM static_datastate_abv",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

static_datastate_abv = download_and_load_query_result(athena_client, response)

new_header = static_datastate_abv.iloc[0] # grab the first row for the header
static_datastate_abv = static_datastate_abv[1:] # take the data less the header row
static_datastate_abv.columns = new_header # set the header row as the new header